#Subway Data

In [2]:
import csv
import datetime
import os
import pandas as pd

from sets import Set
from dateutil.parser import parse
import matplotlib.pyplot as plt

import requests
import urllib2

from bs4 import BeautifulSoup as bs

%matplotlib inline
pd.options.display.mpl_style = 'default'

## Single File

### Quick Overview

Firstly, I'm going to load in one file from an earlier that I've manually downloaded from the MTA website at http://web.mta.info/developers/turnstile.html. After I get a sense of one subset of the data and how to clean it, I'll make a script to download all of the data files, clean and combine them.

In [73]:
filename = "data/turnstile/turnstile_110528.txt"
df_turnstile = pd.read_csv(filename)
df_turnstile.head(2)

,A002,R051,02-00-00,05-21-11,00:00:00,REGULAR,003169391,001097585,05-21-11.1,04:00:00,...,05-22-11,00:00:00.1,REGULAR.6,003170119,001097792,05-22-11.1,04:00:00.1,REGULAR.7,003170146,001097801
0,A002,R051,02-00-00,05-22-11,08:00:00,REGULAR,3170164,1097820,05-22-11,12:00:00,...,05-23-11,08:00:00,REGULAR,3170746,1098069,05-23-11,12:00:00,REGULAR,3170897,1098378
1,A002,R051,02-00-00,05-23-11,16:00:00,REGULAR,3171194,1098447,05-23-11,20:00:00,...,05-24-11,16:00:00,REGULAR,3172689,1099010,05-24-11,20:00:00,REGULAR,3173590,1099055


In [78]:
df_turnstile.shape

(998, 43)

There are 998 rows and an overwhelming 43 columns in this dataset. This file was written in a way where multiple observations share the same row. As a result the MTA data is notoriously difficult to work with. As adaptable as the pandas module is, it can't infer this kind of error and correct it. So I'll have to manually do it myself.

Looking at the time and date columns, it's clear that each file spans exactly one week at 4 hour intervals. The time diference between each consecutive row is 32 hours.

The MTA website labels the data as follows: 'C/A, UNIT, SCP, DATEn, TIMEn, DESCn, ENTRIESn, EXITSn'. The first 3 columns are identification data. Then elements should be chopped from the original data, 5 elements at a time and written into the new file. Next is a sequence of columns with a timestamp, type of report, entry count, and exit count, which repeats 8 times! This figure lines up with the shape of our dataframe (3 + (5 x 8)) = 43.



### Restructuring the data

In [20]:
def fix_turnstile_data(filepath):
    '''
    Filepath is a location of a MTA Subway turnstile text file.A link to an example
    MTA Subway turnstile text file can be seen at the URL below:
    http://web.mta.info/developers/data/nyct/turnstile/turnstile_110507.txt
    
    There are numerous data points included in each row of the text file. 

    This function updates each row in the text file so there is only one entry per row.
    A few examples below:
    A002,R051,02-00-00,05-28-11,00:00:00,REGULAR,003178521,001100739
    A002,R051,02-00-00,05-28-11,04:00:00,REGULAR,003178541,001100746
    A002,R051,02-00-00,05-28-11,08:00:00,REGULAR,003178559,001100775
    
    This file is then written into a new related directory.
    '''
    
    for file in filepath:
        
        # Parse the directory and filename from the input.
        splitted = file.split('/')
        directory, filename = splitted[0:2], splitted[2]

        # Read the file into memory.
        r = csv.reader(open(file, 'rb'))

        # Prepare the output directory.
        newpath = "data/turnstile/updated_data"
        #newpath = 'data/turnstile'
        
        if not os.path.exists(newpath): 
            os.makedirs(newpath)

        # Create the output file in the new directory. Overwrite the file if it exists already(wb).
        w = csv.writer(open("{0}/{1}".format(newpath, filename), 'wb'))

        # Write the header row, taken from the mta website.
        w.writerow(['C/A', 'UNIT', 'SCP', 'DATEn', 'TIMEn', 'DESCn', 'ENTRIESn', 'EXITSn'])

        # Loop through the output from the CSV reader a line at a time.
        for line in r:

            # Parse out the elements, and remove them from the row.
            ca = line.pop(0)
            unit = line.pop(0)
            scp = line.pop(0)

            # While there is still new data, parse it.
            while len(line) >= 5:

                # Take the first 5 elements and remove them.
                block, line = line[:5], line[5:]

                # Output the new row.
                w.writerow([ca, unit, scp] + block)

As it turns out, the MTA have been overhauling their data collection. Their website, API and even the data recording method have changed. The new files are all structured nicely, meaning that the previous work was unneccessary. Oh well, it was good data munging practice at least!

In [2]:
# Load in a new, recent record.
newfile = 'data/turnstile/150502.txt'

#df = pd.read_csv(newfile, sep=r"\s+")
#df = pd.read_csv(newfile, skipinitialspace=True)
df = pd.read_csv(newfile)
df.head(2)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,00:00:00,REGULAR,5106770,1729635
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,REGULAR,5106810,1729649


In [3]:
df.columns.values

array(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE',
       'TIME', 'DESC', 'ENTRIES',
       'EXITS                                                               '], dtype=object)

There's trailing white space after EXITS, need to manually deal with it.

In [4]:
df.columns = ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES', 'EXITS']

### Observations not on the Hour

In [5]:
filter_onhour = [((pd.to_datetime(df.TIME[n])).minute != 0) for n in range(len(df))]

In [6]:
df[filter_onhour].head(2)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
1707,A013,R081,01-00-00,49 ST-7 AVE,NQR,BMT,04/29/2015,08:12:40,REGULAR,5270306,30939048
1750,A013,R081,01-03-00,49 ST-7 AVE,NQR,BMT,04/29/2015,08:12:40,REGULAR,2911534,3345354


In [7]:
df.ix[[1706, 1707, 1708]]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
1706,A013,R081,01-00-00,49 ST-7 AVE,NQR,BMT,04/29/2015,08:00:00,REGULAR,5270301,30938948
1707,A013,R081,01-00-00,49 ST-7 AVE,NQR,BMT,04/29/2015,08:12:40,REGULAR,5270306,30939048
1708,A013,R081,01-00-00,49 ST-7 AVE,NQR,BMT,04/29/2015,12:00:00,REGULAR,5270372,30940882


In [8]:
df2 = df.drop(list(df.loc[filter_onhour].index))

In [9]:
df2 = df2.reset_index()

In [10]:
df = df2
del df['index']

In [11]:
df.ix[[1706]]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
1706,A013,R081,01-00-00,49 ST-7 AVE,NQR,BMT,04/29/2015,08:00:00,REGULAR,5270301,30938948


###Entries and Exits

In [12]:
df[df.ENTRIES == 0].head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
420,A002,R051,02-05-01,LEXINGTON AVE,NQR456,BMT,04/25/2015,00:00:00,REGULAR,0,593
421,A002,R051,02-05-01,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,REGULAR,0,593
422,A002,R051,02-05-01,LEXINGTON AVE,NQR456,BMT,04/25/2015,08:00:00,REGULAR,0,593
423,A002,R051,02-05-01,LEXINGTON AVE,NQR456,BMT,04/25/2015,12:00:00,REGULAR,0,593
424,A002,R051,02-05-01,LEXINGTON AVE,NQR456,BMT,04/25/2015,16:00:00,REGULAR,0,593


In [13]:
#df.groupby(['SCP', 'DATE']).plot(x='TIME', y='ENTRIES')
#plt.show()

####Taking One Day

In [198]:
df1 = df[df.DATE == '05/01/2015']

In [199]:
df1

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
36,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,00:00:00,REGULAR,5115461,1732389
37,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,04:00:00,REGULAR,5115480,1732394
38,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,08:00:00,REGULAR,5115524,1732482
39,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,12:00:00,REGULAR,5115678,1732624
40,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,16:00:00,REGULAR,5115998,1732647
41,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,20:00:00,REGULAR,5116883,1732666
78,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,00:00:00,REGULAR,4738746,1034160
79,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,04:00:00,REGULAR,4738761,1034161
80,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,08:00:00,REGULAR,4738804,1034199
81,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,12:00:00,REGULAR,4738969,1034274


From the documentation on [MTA](http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description.txt), here's what the variables mean:



````
======================================
Field Description

C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS


C/A      = Control Area (A002) - Booth
UNIT     = Remote Unit for a station (R051)
SCP      = Subunit Channel Position represents an specific address for a device (02-00-00)
STATION  = Represents the station name the device is located at
LINENAME = Represents all train lines that can be boarded at this station
           Normally lines are represented by one character.  LINENAME 456NQR repersents train server for 4, 5, 6, N, Q, and R trains.
DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND   
DATE     = Represents the date (MM-DD-YY)
TIME     = Represents the time (hh:mm:ss) for a scheduled audit event
DESc     = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)
           1. Audits may occur more that 4 hours due to planning, or troubleshooting activities. 
           2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered. 
ENTRIES  = The comulative entry register value for a device
EXIST    = The cumulative exit register value for a device



Example:
The data below shows the entry/exit register values for one turnstile at control area (A002) from 09/27/14 at 00:00 hours to 09/29/14 at 00:00 hours


C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-27-14,00:00:00,REGULAR,0004800073,0001629137,
A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-27-14,04:00:00,REGULAR,0004800125,0001629149,
A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-27-14,08:00:00,REGULAR,0004800146,0001629162,
A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-27-14,12:00:00,REGULAR,0004800264,0001629264,
A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-27-14,16:00:00,REGULAR,0004800523,0001629328,
A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-27-14,20:00:00,REGULAR,0004800924,0001629371,
A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-28-14,00:00:00,REGULAR,0004801104,0001629395,
A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-28-14,04:00:00,REGULAR,0004801149,0001629402,
A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-28-14,08:00:00,REGULAR,0004801168,0001629414,
A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-28-14,12:00:00,REGULAR,0004801304,0001629463,
A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-28-14,16:00:00,REGULAR,0004801463,0001629521,
A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-28-14,20:00:00,REGULAR,0004801737,0001629555,
A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-29-14,00:00:00,REGULAR,0004801836,0001629574,
======================================
````

In [50]:
df1.groupby(['C/A', 'UNIT', 'SCP']).first()

STATION LINENAME DIVISION        DATE      TIME  \
C/A   UNIT SCP                                                               
A002  R051 02-00-00  LEXINGTON AVE   NQR456      BMT  05/01/2015  00:00:00   
           02-00-01  LEXINGTON AVE   NQR456      BMT  05/01/2015  00:00:00   
           02-03-00  LEXINGTON AVE   NQR456      BMT  05/01/2015  00:00:00   
           02-03-01  LEXINGTON AVE   NQR456      BMT  05/01/2015  00:00:00   
           02-03-02  LEXINGTON AVE   NQR456      BMT  05/01/2015  00:00:00   
           02-03-03  LEXINGTON AVE   NQR456      BMT  05/01/2015  00:00:00   
           02-03-04  LEXINGTON AVE   NQR456      BMT  05/01/2015  00:00:00   
           02-03-05  LEXINGTON AVE   NQR456      BMT  05/01/2015  00:00:00   
           02-03-06  LEXINGTON AVE   NQR456      BMT  05/01/2015  00:00:00   
           02-05-00  LEXINGTON AVE   NQR456      BMT  05/01/2015  00:00:00   
           02-05-01  LEXINGTON AVE   NQR456      BMT  05/01/2015  00:00:00   
           02-06-00  LEXINGTON AVE   NQR456      BMT  05/01/2015  00:00:00   
A006  R079 00-00-00    5 AVE-59 ST      NQR      BMT  05/01/2015  02:00:00   
           00-00-01    5 AVE-59 ST      NQR      BMT  05/01/2015  02:00:00   
           00-00-02    5 AVE-59 ST      NQR      BMT  05/01/2015  02:00:00   
           00-00-03    5 AVE-59 ST      NQR      BMT  05/01/2015  02:00:00   
           00-00-04    5 AVE-59 ST      NQR      BMT  05/01/2015  02:00:00   
           00-03-00    5 AVE-59 ST      NQR      BMT  05/01/2015  02:00:00   
           00-03-01    5 AVE-59 ST      NQR      BMT  05/01/2015  02:00:00   
           00-03-02    5 AVE-59 ST      NQR      BMT  05/01/2015  02:00:00   
A007  R079 01-05-00    5 AVE-59 ST      NQR      BMT  05/01/2015  02:00:00   
           01-05-01    5 AVE-59 ST      NQR      BMT  05/01/2015  02:00:00   
           01-06-00    5 AVE-59 ST      NQR      BMT  05/01/2015  02:00:00   
           01-06-01    5 AVE-59 ST      NQR      BMT  05/01/2015  02:00:00   
           01-06-02    5 AVE-59 ST      NQR      BMT  05/01/2015  02:00:00   
           01-06-03    5 AVE-59 ST      NQR      BMT  05/01/2015  02:00:00   
A010  R080 00-00-00    57 ST-7 AVE      NQR      BMT  05/01/2015  00:00:00   
           00-00-01    57 ST-7 AVE      NQR      BMT  05/01/2015  00:00:00   
           00-00-02    57 ST-7 AVE      NQR      BMT  05/01/2015  00:00:00   
           00-00-03    57 ST-7 AVE      NQR      BMT  05/01/2015  00:00:00   
...                            ...      ...      ...         ...       ...   
S101A R070 01-00-00     ST. GEORGE        1      SRT  05/01/2015  00:00:00   
           01-00-01     ST. GEORGE        1      SRT  05/01/2015  00:00:00   
           01-00-02     ST. GEORGE        1      SRT  05/01/2015  00:00:00   
           01-00-03     ST. GEORGE        1      SRT  05/01/2015  00:00:00   
           01-00-04     ST. GEORGE        1      SRT  05/01/2015  00:00:00   
           01-00-05     ST. GEORGE        1      SRT  05/01/2015  00:00:00   
           01-00-06     ST. GEORGE        1      SRT  05/01/2015  00:00:00   
           01-03-00     ST. GEORGE        1      SRT  05/01/2015  00:00:00   
           01-03-01     ST. GEORGE        1      SRT  05/01/2015  00:00:00   
           01-03-02     ST. GEORGE        1      SRT  05/01/2015  00:00:00   
           01-03-03     ST. GEORGE        1      SRT  05/01/2015  00:00:00   
           01-05-00     ST. GEORGE        1      SRT  05/01/2015  00:00:00   
           01-05-01     ST. GEORGE        1      SRT  05/01/2015  00:00:00   
S102  R165 00-00-00  TOMPKINSVILLE        1      SRT  05/01/2015  02:00:00   
           00-00-01  TOMPKINSVILLE        1      SRT  05/01/2015  02:00:00   
           00-03-00  TOMPKINSVILLE        1      SRT  05/01/2015  02:00:00   
           00-03-01  TOMPKINSVILLE        1      SRT  05/01/2015  02:00:00   
           00-03-02  TOMPKINSVILLE        1      SRT  05/01/2015  02:00:00   
           00-05-00  TOMPKINSVILLE        1      

In [51]:
df1.groupby(['STATION', 'C/A', 'UNIT', 'SCP']).first()

LINENAME DIVISION        DATE      TIME  \
STATION         C/A   UNIT SCP                                                
1 AVE           H007  R248 00-00-00        L      BMT  05/01/2015  00:00:00   
                           00-00-01        L      BMT  05/01/2015  00:00:00   
                           00-03-00        L      BMT  05/01/2015  00:00:00   
                           00-03-01        L      BMT  05/01/2015  00:00:00   
                           00-03-02        L      BMT  05/01/2015  00:00:00   
                H008  R248 01-00-00        L      BMT  05/01/2015  00:00:00   
                           01-00-01        L      BMT  05/01/2015  00:00:00   
                           01-00-02        L      BMT  05/01/2015  00:00:00   
                           01-00-03        L      BMT  05/01/2015  00:00:00   
                           01-00-04        L      BMT  05/01/2015  00:00:00   
103 ST          N037  R314 00-00-00       BC      IND  05/01/2015  01:00:00   
                           00-00-01       BC      IND  05/01/2015  01:00:00   
                           00-00-02       BC      IND  05/01/2015  01:00:00   
                R170  R191 00-00-00        1      IRT  05/01/2015  01:00:00   
                           00-00-01        1      IRT  05/01/2015  01:00:00   
                           00-00-02        1      IRT  05/01/2015  01:00:00   
                           00-03-00        1      IRT  05/01/2015  01:00:00   
                           00-03-01        1      IRT  05/01/2015  01:00:00   
                           00-03-02        1      IRT  05/01/2015  01:00:00   
                R252  R180 00-00-00        6      IRT  05/01/2015  01:00:00   
                           00-00-01        6      IRT  05/01/2015  01:00:00   
                           00-00-02        6      IRT  05/01/2015  01:00:00   
                           00-00-03        6      IRT  05/01/2015  01:00:00   
                           00-00-04        6      IRT  05/01/2015  01:00:00   
                           00-00-05        6      IRT  05/01/2015  01:00:00   
103 ST-CORONA   R529  R208 00-00-00        7      IRT  05/01/2015  00:00:00   
                           00-00-01        7      IRT  05/01/2015  00:00:00   
                           00-00-02        7      IRT  05/01/2015  00:00:00   
                           00-00-03        7      IRT  05/01/2015  00:00:00   
                           00-00-04        7      IRT  05/01/2015  00:00:00   
...                                      ...      ...         ...       ...   
WOODHAVEN BLVD  N329  R201 00-03-04       MR      IND  05/01/2015  01:00:00   
                N329A R201 01-05-00       MR      IND  05/01/2015  01:00:00   
                           01-05-01       MR      IND  05/01/2015  01:00:00   
                           01-06-00       MR      IND  05/01/2015  01:00:00   
                           01-06-01       MR      IND  05/01/2015  01:00:00   
WOODLAWN ROAD   R294  R052 00-00-00        4      IRT  05/01/2015  00:00:00   
                           00-00-01        4      IRT  05/01/2015  00:00:00   
                           00-00-02        4      IRT  05/01/2015  00:00:00   
                           00-00-03        4      IRT  05/01/2015  00:00:00   
                           00-00-04        4      IRT  05/01/2015  00:00:00   
WORLD TRADE CTR N094  R029 01-00-00    ACE23      IND  05/01/2015  00:00:00   
                           01-00-01    ACE23      IND  05/01/2015  00:00:00   
                           01-00-02    ACE23      IND  05/01/2015  00:00:00   
                           01-00-03    ACE23      IND  05/01/2015  00:00:00   
                           01-05-00    ACE23      IND  05/01/2015  00:00:00   
                           01-05-01    ACE23      IND  05/01/2015  00:00:00   
                           01-06-00    ACE23      IND  05/01/2015  00:00:00   
                           01-06-01    ACE23      IND  05/01/2015  00:00:00   
       

Basically, for each identifier ('C/A', 'UNIT', 'SCP') I want to have hourly entries and exits that are initialized at 0 at the start of the day (00:00:00) and are updated on 4 hour intervals by the difference of the cumulative values.

In [52]:
filter_first = (df1['C/A'] == 'A002') & (df1['UNIT'] == 'R051') & (df1['SCP'] == '02-00-00')

In [53]:
df10 = df1[filter_first]
df10 = df1.reset_index()
del df10['index']

In [54]:
df10['ENTRIES_hourly'] = df10['ENTRIES'] - df10['ENTRIES'].shift(1)
df10['ENTRIES_hourly'] = df10['ENTRIES_hourly'].fillna(0)

df10['EXITS_hourly'] = df10['EXITS'] - df10['EXITS'].shift(1)
df10['EXITS_hourly'] = df10['EXITS_hourly'].fillna(0)

In [55]:
df10.head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,ENTRIES_hourly,EXITS_hourly
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,00:00:00,REGULAR,5115461,1732389,0,0
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,04:00:00,REGULAR,5115480,1732394,19,5
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,08:00:00,REGULAR,5115524,1732482,44,88
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,12:00:00,REGULAR,5115678,1732624,154,142
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,16:00:00,REGULAR,5115998,1732647,320,23
5,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,20:00:00,REGULAR,5116883,1732666,885,19
6,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,00:00:00,REGULAR,4738746,1034160,-378137,-698506
7,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,04:00:00,REGULAR,4738761,1034161,15,1
8,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,08:00:00,REGULAR,4738804,1034199,43,38
9,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,12:00:00,REGULAR,4738969,1034274,165,75


Ok, it worked for one turnstile on one date. Now it needs to work on them ALL on that date, and then on ALL turnstiles on ALL of the dates.

In [5]:
=================================================================================

SyntaxError: invalid syntax (<ipython-input-5-871e92fec658>, line 1)

In [26]:
# Load in a new, recent record.
newfile = 'data/turnstile/150502.txt'

#df = pd.read_csv(newfile, sep=r"\s+")
#df = pd.read_csv(newfile, skipinitialspace=True)
df1 = pd.read_csv(newfile)
df1.columns = ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES', 'EXITS']
df1.head(2)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,00:00:00,REGULAR,5106770,1729635
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,REGULAR,5106810,1729649


In [27]:
newfile = 'data/turnstile/150509.txt'
df2 = pd.read_csv(newfile)
df2.columns = ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES', 'EXITS']
df2.head(2)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/02/2015,00:00:00,REGULAR,5117130,1732680
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/02/2015,04:00:00,REGULAR,5117157,1732685


It seems like C/A, UNIT and SCP are the 3 identifiers that I need to run through in order to get every turnstile. 

For each turnstile's first entry on May 1st I need to set its entrties and exits to 0. Then calculate the rest of the entries and exits for the day by taking the difference of consecutive ones. Finally for the remaining days I will need to look up what the count was for the last entry on the previous day.

I think a dictionary is the best way to do this.

In [140]:
df_testing= df1[df1['DATE'] == '05/01/2015'][:10]
df_testing = df_testing.reset_index()
df_testing = df_testing.drop('index', axis=1)
df_testing

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,00:00:00,REGULAR,5115461,1732389
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,04:00:00,REGULAR,5115480,1732394
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,08:00:00,REGULAR,5115524,1732482
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,12:00:00,REGULAR,5115678,1732624
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,16:00:00,REGULAR,5115998,1732647
5,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,20:00:00,REGULAR,5116883,1732666
6,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,00:00:00,REGULAR,4738746,1034160
7,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,04:00:00,REGULAR,4738761,1034161
8,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,08:00:00,REGULAR,4738804,1034199
9,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,12:00:00,REGULAR,4738969,1034274


In [143]:
df = df_testing

In [145]:
def cumulative_to_hourly(df):
    for i, entry in df_testing.iterrows():
        id_ = entry['C/A'], entry['UNIT'], entry['SCP']
        print "{} \t {} \t {} \t {}".format(entry['index'], id_, entry['DATE'], entry['TIME'])
        
        
        #If first entry of May 1st.
        if (entry['DATE'] == '05/01/2015' and entry['TIME'] == '00:00:00') :
            
            df.loc[i, 'HOURLY_ENTRIES'] = 0
            df.loc[i, 'HOURLY_EXITS'] = 0
            
        
        # Otherwise, look up previous entry for that turnstile.
        else:
            # Get all entries for that turnstile
            previous_entries = df[(df['C/A']==('A002')) & (df['UNIT']==('R051')) & (df['SCP']==('02-00-00')) & (df.index < i)]
            j = max(previous_entries.index)
            #print previous_entries
            print i,j,previous_entries.index
        
            
            df.loc[i, 'HOURLY_ENTRIES'] = df.loc[i, 'ENTRIES'] - df.loc[j, 'ENTRIES']
            df.loc[i, 'HOURLY_EXITS'] = df.loc[i, 'EXITS'] - df.loc[j, 'EXITS']

In [146]:
cumulative_to_hourly(df_testing)
df_testing

KeyError: 'index'

That worked. Now time to do it with multiple dates.

In [277]:
df1 = df1[df1.DATE == '05/01/2015']
df2 = df2[df2.DATE == '05/02/2015']
df_testing_2 = pd.concat([df1[:3], df2[:3]])
df_testing_2 = df_testing_2.reset_index()

df_testing_2

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,36,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,00:00:00,REGULAR,5115461,1732389
1,37,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,04:00:00,REGULAR,5115480,1732394
2,38,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,08:00:00,REGULAR,5115524,1732482
3,0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/02/2015,00:00:00,REGULAR,5117130,1732680
4,1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/02/2015,04:00:00,REGULAR,5117157,1732685
5,2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/02/2015,08:00:00,REGULAR,5117176,1732693


In [16]:
df1.loc[0,"ENTRIES_HOURLY"] = "asd"

In [17]:
df1

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,ENTRIES_HOURLY
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,00:00:00,REGULAR,5106770,1729635,asd
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,REGULAR,5106810,1729649,NaN
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,08:00:00,REGULAR,5106835,1729680,NaN
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,12:00:00,REGULAR,5106961,1729784,NaN
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,16:00:00,REGULAR,5107250,1729858,NaN
5,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,20:00:00,REGULAR,5107620,1729914,NaN
6,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/26/2015,00:00:00,REGULAR,5107793,1729955,NaN
7,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/26/2015,04:00:00,REGULAR,5107815,1729964,NaN
8,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/26/2015,08:00:00,REGULAR,5107844,1729991,NaN
9,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/26/2015,12:00:00,REGULAR,5107912,1730068,NaN


In [278]:
cumulative_to_hourly(df_testing_2)

A002-R051-02-00-00 	 05/01/2015 	 00:00:00
HOURLY ENTRIES = 	 0 	 0
CUMULATIVE ENTRIES = [0L, 0L]

A002-R051-02-00-00 	 05/01/2015 	 04:00:00
HOURLY ENTRIES = 	 19 	 5
CUMULATIVE ENTRIES = [19L, 5L]

A002-R051-02-00-00 	 05/01/2015 	 08:00:00
HOURLY ENTRIES = 	 6 	 78
CUMULATIVE ENTRIES = [25L, 83L]

A002-R051-02-00-00 	 05/01/2015 	 12:00:00
HOURLY ENTRIES = 	 85 	 -29
CUMULATIVE ENTRIES = [110L, 54L]

A002-R051-02-00-00 	 05/01/2015 	 16:00:00
HOURLY ENTRIES = 	 56 	 -173
CUMULATIVE ENTRIES = [166L, -119L]

A002-R051-02-00-00 	 05/01/2015 	 20:00:00
HOURLY ENTRIES = 	 399 	 115
CUMULATIVE ENTRIES = [565L, -4L]

A002-R051-02-00-01 	 05/01/2015 	 00:00:00
HOURLY ENTRIES = 	 0.0 	 0.0
CUMULATIVE ENTRIES = [0L, 0L]

A002-R051-02-00-01 	 05/01/2015 	 04:00:00
HOURLY ENTRIES = 	 15.0 	 1.0
CUMULATIVE ENTRIES = [15L, 1L]

A002-R051-02-00-01 	 05/01/2015 	 08:00:00
HOURLY ENTRIES = 	 13.0 	 36.0
CUMULATIVE ENTRIES = [28L, 37L]

A002-R051-02-00-01 	 05/01/2015 	 12:00:00
HOURLY ENTRIES = 	 94

{'A002-R051-02-00-00': [565L, -4L],
 'A002-R051-02-00-01': [451L, -15L],
 'A002-R051-02-03-00': [244L, 80L],
 'A002-R051-02-03-01': [36L, 14L]}

In [276]:
df_testing_2

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,36,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,00:00:00,REGULAR,0,0
1,37,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,04:00:00,REGULAR,19,5
2,38,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,08:00:00,REGULAR,6,78
3,0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/02/2015,00:00:00,REGULAR,85,-29
4,1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/02/2015,04:00:00,REGULAR,56,-173
5,2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/02/2015,08:00:00,REGULAR,399,115
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,1
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,36
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94,0


In [248]:
df_testing_2[(df_testing_2.DATE == ('05/01/2015')) | (df_testing_2.DATE == ('05/02/2015'))]

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,36,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,00:00:00,REGULAR,5115461,1732389
1,37,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,04:00:00,REGULAR,5115480,1732394
2,38,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,08:00:00,REGULAR,5115524,1732482
3,39,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,12:00:00,REGULAR,5115678,1732624
4,40,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,16:00:00,REGULAR,5115998,1732647
5,41,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,20:00:00,REGULAR,5116883,1732666
6,78,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,00:00:00,REGULAR,4738746,1034160
7,79,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,04:00:00,REGULAR,4738761,1034161
8,80,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,08:00:00,REGULAR,4738804,1034199
9,81,A002,R051,02-00-01,LEXINGTON AVE,NQR456,BMT,05/01/2015,12:00:00,REGULAR,4738969,1034274


### Different Types

There are different types of "DESC" according to the documentation.

===========================
````
Descn Possible Values (Events):
REGULAR - Regular scheduled audit event
NO-VAL LGN - Not Valid logon
LGF-MAN - Logoff Manual
LGF-DR CLS - Logoff Door Closed
LGF-SHUTDN - Logoff Shutdown
TS BRD CHG - Turnstile Board Change
TS VLT OPN - Turnstile Vault Open
RECOVR AUD - Recovery audit - if REGULAR was not delivered due to communications problems
````
===========================


In [216]:
df.DESC.unique()

array(['REGULAR', 'RECOVR AUD'], dtype=object)

In [224]:
df[df.DESC == 'RECOVR AUD'].head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
1113,A010,R080,00-00-00,57 ST-7 AVE,NQR,BMT,04/28/2015,12:00:00,RECOVR AUD,143112,85613
1155,A010,R080,00-00-01,57 ST-7 AVE,NQR,BMT,04/28/2015,12:00:00,RECOVR AUD,12250490,4537590
1197,A010,R080,00-00-02,57 ST-7 AVE,NQR,BMT,04/28/2015,12:00:00,RECOVR AUD,8936510,3456835
1239,A010,R080,00-00-03,57 ST-7 AVE,NQR,BMT,04/28/2015,12:00:00,RECOVR AUD,1865400,918583
1281,A010,R080,00-00-04,57 ST-7 AVE,NQR,BMT,04/28/2015,12:00:00,RECOVR AUD,2550498,1228635


In [222]:
df.iloc[[1110, 1112, 1113, 1114, 1115]]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
1110,A010,R080,00-00-00,57 ST-7 AVE,NQR,BMT,04/28/2015,00:00:00,REGULAR,142599,84827
1112,A010,R080,00-00-00,57 ST-7 AVE,NQR,BMT,04/28/2015,08:00:00,REGULAR,142744,85024
1113,A010,R080,00-00-00,57 ST-7 AVE,NQR,BMT,04/28/2015,12:00:00,RECOVR AUD,143112,85613
1114,A010,R080,00-00-00,57 ST-7 AVE,NQR,BMT,04/28/2015,16:00:00,REGULAR,143638,85933
1115,A010,R080,00-00-00,57 ST-7 AVE,NQR,BMT,04/28/2015,20:00:00,REGULAR,144687,86304


In this dataset there are only two different types, REGULAR and RECOVR AUD. Fortunately RECOVR AUD does not require any further thought as it gives the same reading as REGULAR, but as a backup communication.

In [23]:
df_turnstile.shape

(7343, 8)

###Checking Udacity

In [28]:
uda = pd.read_csv('data/turnstile_data_master_with_weather.csv')

In [29]:
uda.shape

(131951, 22)

In [30]:
uda.head(100)

,Unnamed: 0,UNIT,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,...,meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder
0,0,R001,2011-05-01,01:00:00,1,REGULAR,0,0,30.31,42,...,39,30.27,0,0,5,50,60,69,0,0
1,1,R001,2011-05-01,05:00:00,5,REGULAR,217,553,30.31,42,...,39,30.27,0,0,5,50,60,69,0,0
2,2,R001,2011-05-01,09:00:00,9,REGULAR,890,1262,30.31,42,...,39,30.27,0,0,5,50,60,69,0,0
3,3,R001,2011-05-01,13:00:00,13,REGULAR,2451,3708,30.31,42,...,39,30.27,0,0,5,50,60,69,0,0
4,4,R001,2011-05-01,17:00:00,17,REGULAR,4400,2501,30.31,42,...,39,30.27,0,0,5,50,60,69,0,0
5,5,R001,2011-05-01,21:00:00,21,REGULAR,3372,2122,30.31,42,...,39,30.27,0,0,5,50,60,69,0,0
6,6,R002,2011-05-01,01:00:00,1,REGULAR,0,0,30.31,42,...,39,30.27,0,0,5,50,60,69,0,0
7,7,R002,2011-05-01,05:00:00,5,REGULAR,42,66,30.31,42,...,39,30.27,0,0,5,50,60,69,0,0
8,8,R002,2011-05-01,09:00:00,9,REGULAR,50,125,30.31,42,...,39,30.27,0,0,5,50,60,69,0,0
9,9,R002,2011-05-01,13:00:00,13,REGULAR,316,716,30.31,42,...,39,30.27,0,0,5,50,60,69,0,0


In [34]:
uda[(uda.DATEn == '2011-05-01') & (uda.UNIT == "R001")]

,Unnamed: 0,UNIT,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,...,meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder
4433,4433,R001,2011-05-02,01:00:00,1,REGULAR,804,1042,30.31,50,...,45,30.22,0,0,5,50,57,63,0,0
4434,4434,R001,2011-05-02,05:00:00,5,REGULAR,175,235,30.31,50,...,45,30.22,0,0,5,50,57,63,0,0
4435,4435,R001,2011-05-02,09:00:00,9,REGULAR,6909,6451,30.31,50,...,45,30.22,0,0,5,50,57,63,0,0
4436,4436,R001,2011-05-02,13:00:00,13,REGULAR,4857,8294,30.31,50,...,45,30.22,0,0,5,50,57,63,0,0
4437,4437,R001,2011-05-02,17:00:00,17,REGULAR,7030,5390,30.31,50,...,45,30.22,0,0,5,50,57,63,0,0
4438,4438,R001,2011-05-02,21:00:00,21,REGULAR,9192,5886,30.31,50,...,45,30.22,0,0,5,50,57,63,0,0


In [35]:
uda[(uda.DATEn == '2011-05-02') & (uda.UNIT == "R001")]

,Unnamed: 0,UNIT,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,...,meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder
4433,4433,R001,2011-05-02,01:00:00,1,REGULAR,804,1042,30.31,50,...,45,30.22,0,0,5,50,57,63,0,0
4434,4434,R001,2011-05-02,05:00:00,5,REGULAR,175,235,30.31,50,...,45,30.22,0,0,5,50,57,63,0,0
4435,4435,R001,2011-05-02,09:00:00,9,REGULAR,6909,6451,30.31,50,...,45,30.22,0,0,5,50,57,63,0,0
4436,4436,R001,2011-05-02,13:00:00,13,REGULAR,4857,8294,30.31,50,...,45,30.22,0,0,5,50,57,63,0,0
4437,4437,R001,2011-05-02,17:00:00,17,REGULAR,7030,5390,30.31,50,...,45,30.22,0,0,5,50,57,63,0,0
4438,4438,R001,2011-05-02,21:00:00,21,REGULAR,9192,5886,30.31,50,...,45,30.22,0,0,5,50,57,63,0,0


In [110]:
a.groupby(['UNIT']).first()

,Unnamed: 0,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,mindewpti,...,meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder
UNIT,,,,,,,,,,,,,,,,,,,,,
R001,0,2011-05-01,01:00:00,1,REGULAR,0,0,30.31,42,35,...,39,30.27,0,0,5,50,60,69,0,0
R002,6,2011-05-01,01:00:00,1,REGULAR,0,0,30.31,42,35,...,39,30.27,0,0,5,50,60,69,0,0
R003,12,2011-05-01,00:00:00,0,REGULAR,0,0,30.31,42,35,...,39,30.27,0,0,5,50,60,69,0,0
R004,17,2011-05-01,00:00:00,0,REGULAR,0,0,30.31,42,35,...,39,30.27,0,0,5,50,60,69,0,0
R005,23,2011-05-01,00:00:00,0,REGULAR,0,0,30.31,42,35,...,39,30.27,0,0,5,50,60,69,0,0
R006,28,2011-05-01,00:00:00,0,REGULAR,0,0,30.31,42,35,...,39,30.27,0,0,5,50,60,69,0,0
R007,34,2011-05-01,00:00:00,0,REGULAR,0,0,30.31,42,35,...,39,30.27,0,0,5,50,60,69,0,0
R008,38,2011-05-01,00:00:00,0,REGULAR,0,0,30.31,42,35,...,39,30.27,0,0,5,50,60,69,0,0
R009,43,2011-05-01,00:00:00,0,REGULAR,0,0,30.31,42,35,...,39,30.27,0,0,5,50,60,69,0,0


## Scraping Files

I'm choosing the last complete month to do analysis on which is May 2015. The MTA collect data weekly, so I'm also going to take the last week of April, and the first month of June.

The files are available for download here: http://web.mta.info/developers/turnstile.html

I'll be using the python module Beautiful Soup to parse the html in order to find the relevant hyperlinks quickly.

There was an odd error with beautiful soup - some byte code on the page couldn't be converted to ASCII characters. After a quick google, I found the following fix on stackoverflow.

In [79]:
# Work around for byte-ASCII error in bs.
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

In [26]:
# URL of MTA data.
URL = 'http://web.mta.info/developers/turnstile.html'

# Parse the html using bs to find the hyperlinks.
r = requests.get(URL)
soup = bs(r.text)
hyperlinks = soup.findAll('a')

# Initialize array to hold all of the URLs on the page.
urls = []

# Loop through the hyperlinks, parsing just the links themselves.
for each in hyperlinks:
    link = each.get('href')
    
    # Only add links which are in the data directory, and are text files.
    if link and link.endswith('.txt') and link.startswith('data/'):
        urls.append(link)

In [80]:
# Initialize array to hold all of the dates from the URLs collected.
dates = []

# Split the URLs up in order to get just the dates.
for each in urls:
    filename = each.split('/')[-1]
    filename.find('turnstile_')
    index = filename.find('turnstile_')
    date = filename[index+len('turnstile_'):].split('.')[0]
    dates.append(date)

In [28]:
# Initialize set to store the dates to download.
dates_to_download = Set([])

# Loop through the dates, and take the appropriate ones.
for i, date in enumerate(dates):
    # Only 2015.
    if date[:2] == '15':
        # Take all of May, the last week of April, and the first week of June.
        if date[2:4] == '05':
            dates_to_download.update([dates[i-1], dates[i], dates[i+1]])

# Convert the set to an array.            
dates_to_download = list(dates_to_download)

In [125]:
dates_to_download = sorted(dates_to_download)
dates_to_download

['150425', '150502', '150509', '150516', '150523', '150530', '150606']

In [15]:
l = len(dates_to_download)

# Loop through the dates, downloading the corresponding file.
for i in range(l):
    date = dates_to_download[i]
    download = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_{0}.txt'.format(date)

## Combining Files

Once all the neccessary subway data has been downloaded, it's time to combine it in some fashion. One possbility is to load all the files into invididual dataframes using pandas, and then merging them and writing. Instead I decided to open the files one by one and write the contents into a new file, as this requires less memory usage.

In [153]:
def combine_turnstile_data(filenames):
    """
    Takes the turnstile filenames and writes them one by one into a new
    file.
    """
    
    # Open a new master file, and write in the header row.
    with open('updated_data/master_file.txt', 'w') as master_file:
        master_file.write('C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATEn,time_n,DESCn,ENTRIESn,EXITSn\n')
    
        # Open all the files and write thme into master file.
        for filename in filenames:
            with open('data/turnstile/{0}.txt'.format(filename),'rb') as f:
                for row in f:
                    # Ignore the the header row.
                    if row.startswith('C/A'):
                        continue
                    master_file.write(row)

In [154]:
combine_turnstile_data(dates_to_download)

In [155]:
df_turnstile_master = pd.read_csv('updated_data/master_file.txt')

In [156]:
df_turnstile_master.head(1)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATEn,time_n,DESCn,ENTRIESn,EXITSn
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/18/2015,00:00:00,REGULAR,5095940,1725998


##Preparing the Data

###Tallying the counts

Additionally the original data set gives running totals for each turnstile instead of just a number of entries or exits, so to get anything useful out of it, I'll need to do some subtraction. Simply subtract the entry tally for one timestamp from the previous reading.

In [157]:
df_turnstile_master['ENTRIESn_hourly'] = df_turnstile_master['ENTRIESn'] - df_turnstile_master['ENTRIESn'].shift(1)
df_turnstile_master['ENTRIESn_hourly'] = df_turnstile_master['ENTRIESn_hourly'].fillna(0)

df_turnstile_master['EXITSn_hourly'] = df_turnstile_master['EXITSn'] - df_turnstile_master['EXITSn'].shift(1)
df_turnstile_master['EXITSn_hourly'] = df_turnstile_master['EXITSn_hourly'].fillna(0)
    
df_turnstile_master.head(3)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATEn,time_n,DESCn,ENTRIESn,EXITSn,ENTRIESn_hourly,EXITSn_hourly
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/18/2015,00:00:00,REGULAR,5095940,1725998,0,0
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/18/2015,04:00:00,REGULAR,5095981,1726007,41,9
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/18/2015,08:00:00,REGULAR,5096001,1726039,20,32


In [182]:
df_turnstile_master[df_turnstile_master.EXITSn_hourly < 0]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,time_n,DESCn,ENTRIESn_hourly,EXITSn_hourly,date_n,weekday
204,A011,R080,01-00-00,57 ST-7 AVE,NQR,BMT,00:00:00,REGULAR,-575,-167,2015-05-01,4
205,A011,R080,01-00-00,57 ST-7 AVE,NQR,BMT,04:00:00,REGULAR,-97,-46,2015-05-01,4
206,A011,R080,01-00-00,57 ST-7 AVE,NQR,BMT,08:00:00,REGULAR,-105,-489,2015-05-01,4
207,A011,R080,01-00-00,57 ST-7 AVE,NQR,BMT,12:00:00,REGULAR,-465,-1293,2015-05-01,4
208,A011,R080,01-00-00,57 ST-7 AVE,NQR,BMT,16:00:00,REGULAR,-639,-449,2015-05-01,4
209,A011,R080,01-00-00,57 ST-7 AVE,NQR,BMT,20:00:00,REGULAR,-1298,-495,2015-05-01,4
222,A011,R080,01-00-03,57 ST-7 AVE,NQR,BMT,00:00:00,REGULAR,252,-158,2015-05-01,4
223,A011,R080,01-00-03,57 ST-7 AVE,NQR,BMT,04:00:00,REGULAR,50,-8,2015-05-01,4
224,A011,R080,01-00-03,57 ST-7 AVE,NQR,BMT,08:00:00,REGULAR,35,-144,2015-05-01,4
225,A011,R080,01-00-03,57 ST-7 AVE,NQR,BMT,12:00:00,REGULAR,200,-563,2015-05-01,4


In [186]:
df_turnstile_master.iloc[[203, 204, 205]]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,time_n,DESCn,ENTRIESn_hourly,EXITSn_hourly,date_n,weekday
203,A010,R080,00-00-07,57 ST-7 AVE,NQR,BMT,20:00:00,REGULAR,1789,79,2015-05-01,4
204,A011,R080,01-00-00,57 ST-7 AVE,NQR,BMT,00:00:00,REGULAR,-575,-167,2015-05-01,4
205,A011,R080,01-00-00,57 ST-7 AVE,NQR,BMT,04:00:00,REGULAR,-97,-46,2015-05-01,4


### Dates

Convert to YYYY-MM-DD format.

In [158]:
# Convert to datetime.
df_turnstile_master['date_n'] = pd.to_datetime(df_turnstile_master['DATEn'])
df_turnstile_master.tail()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATEn,time_n,DESCn,ENTRIESn,EXITSn,ENTRIESn_hourly,EXITSn_hourly,date_n
1343579,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/05/2015,05:00:00,REGULAR,5554,212,0,0,2015-06-05
1343580,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/05/2015,09:00:00,REGULAR,5554,212,0,0,2015-06-05
1343581,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/05/2015,13:00:00,REGULAR,5554,212,0,0,2015-06-05
1343582,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/05/2015,17:00:00,REGULAR,5554,212,0,0,2015-06-05
1343583,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/05/2015,21:00:00,REGULAR,5554,212,0,0,2015-06-05


### Removing Non-May Dates

In [159]:
# Checking if the dates are in order.
pd.Series(df_turnstile_master.date_n.ravel()).unique()

array(['2015-04-18T01:00:00.000000000+0100',
       '2015-04-19T01:00:00.000000000+0100',
       '2015-04-20T01:00:00.000000000+0100',
       '2015-04-21T01:00:00.000000000+0100',
       '2015-04-22T01:00:00.000000000+0100',
       '2015-04-23T01:00:00.000000000+0100',
       '2015-04-24T01:00:00.000000000+0100',
       '2015-04-25T01:00:00.000000000+0100',
       '2015-04-26T01:00:00.000000000+0100',
       '2015-04-27T01:00:00.000000000+0100',
       '2015-04-28T01:00:00.000000000+0100',
       '2015-04-29T01:00:00.000000000+0100',
       '2015-04-30T01:00:00.000000000+0100',
       '2015-05-01T01:00:00.000000000+0100',
       '2015-05-02T01:00:00.000000000+0100',
       '2015-05-03T01:00:00.000000000+0100',
       '2015-05-04T01:00:00.000000000+0100',
       '2015-05-05T01:00:00.000000000+0100',
       '2015-05-06T01:00:00.000000000+0100',
       '2015-05-07T01:00:00.000000000+0100',
       '2015-05-08T01:00:00.000000000+0100',
       '2015-05-09T01:00:00.000000000+0100',
       '20

Now I've got to remove the April and June dates from the data. 

In [160]:
before = df_turnstile_master.shape[0]

In [161]:
# Convert the date_n col to a datetime and extract the month.
month_filter =  pd.DatetimeIndex(df_turnstile_master['date_n']).month

In [162]:
# Convert to a Series in order to do filter operation on the df.
month_filter = pd.Series(month_filter)

In [163]:
# Subset the df, based on the month being May.
df_turnstile_master = df_turnstile_master[month_filter == 5]

In [164]:
after = df_turnstile_master.shape[0]

In [165]:
before

1343584

In [166]:
after

850343

In [167]:
pd.Series(df_turnstile_master.date_n.ravel()).unique()

array(['2015-05-01T01:00:00.000000000+0100',
       '2015-05-02T01:00:00.000000000+0100',
       '2015-05-03T01:00:00.000000000+0100',
       '2015-05-04T01:00:00.000000000+0100',
       '2015-05-05T01:00:00.000000000+0100',
       '2015-05-06T01:00:00.000000000+0100',
       '2015-05-07T01:00:00.000000000+0100',
       '2015-05-08T01:00:00.000000000+0100',
       '2015-05-09T01:00:00.000000000+0100',
       '2015-05-10T01:00:00.000000000+0100',
       '2015-05-11T01:00:00.000000000+0100',
       '2015-05-12T01:00:00.000000000+0100',
       '2015-05-13T01:00:00.000000000+0100',
       '2015-05-14T01:00:00.000000000+0100',
       '2015-05-15T01:00:00.000000000+0100',
       '2015-05-16T01:00:00.000000000+0100',
       '2015-05-17T01:00:00.000000000+0100',
       '2015-05-18T01:00:00.000000000+0100',
       '2015-05-19T01:00:00.000000000+0100',
       '2015-05-20T01:00:00.000000000+0100',
       '2015-05-21T01:00:00.000000000+0100',
       '2015-05-22T01:00:00.000000000+0100',
       '20

In [168]:
df_turnstile_master.head(1)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATEn,time_n,DESCn,ENTRIESn,EXITSn,ENTRIESn_hourly,EXITSn_hourly,date_n
191038,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,05/01/2015,00:00:00,REGULAR,5115461,1732389,242,27,2015-05-01


### Removing Redundant Columns

In [169]:
df_turnstile_master = df_turnstile_master.drop(["DATEn", "ENTRIESn", "EXITSn"], axis=1)

In [170]:
df_turnstile_master.shape

(850343, 11)

###Adding Weekday

In [172]:
df_turnstile_master['weekday'] = df_turnstile_master['date_n'].apply(lambda x: x.weekday())

In [178]:
# Reset the index.
df_turnstile_master = df_turnstile_master.reset_index()

del df_turnstile_master['index']

##Exporting Final File

In [180]:
df_turnstile_master.to_csv('data/final/turnstile_final.csv', index=False, date_format='%Y-%m-%d')

In [179]:
df_turnstile_master.head(1)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,time_n,DESCn,ENTRIESn_hourly,EXITSn_hourly,date_n,weekday
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,00:00:00,REGULAR,242,27,2015-05-01,4


### Using Both Datasets

I thought about combining both dataframes into a master dataset that would contain both subway and weather information. However after I implemented it, the whole thing proved to be redundant. The weather data contains one observation for the entire day, while the subway data contains data for every 4 hours. Combining them would mean putting a lot of repeated data into each hour for each day. This only offers some ease of access by having everything in one file, but it bloats the data. So instead I'm choosing to keep them separate, and keeping the weather dataframe as a sort of lookup table to find dates corresponding to events (such as fog) and then getting the subway data for these dates.

But first I'll index them both by date to align them and make it easier for lookups.

In [184]:
df_weather = df_weather.set_index('date')
df_turnstile_master = df_turnstile_master.set_index('date')

KeyError: 'date'

In [19]:
filter_fog = df_weather['fog'] == 1
a = df_weather[filter_fog]
a.index


Index([u'2011-05-15', u'2011-05-18', u'2011-05-19', u'2011-05-23', u'2011-05-24'], dtype='object')

In [20]:
df_turnstile['2011-05-21']

,C/A,UNIT,SCP,TIMEn,DESCn,ENTRIESn_hourly,EXITSn_hourly
date,,,,,,,
2011-05-21,A002,R051,02-00-00,00:00:00,REGULAR,0,0
2011-05-21,A002,R051,02-00-00,04:00:00,REGULAR,24,3
2011-05-21,A002,R051,02-00-00,08:00:00,REGULAR,16,19
2011-05-21,A002,R051,02-00-00,12:00:00,REGULAR,75,79
2011-05-21,A002,R051,02-00-00,16:00:00,REGULAR,187,48
2011-05-21,A002,R051,02-00-00,20:00:00,REGULAR,305,35
2011-05-21,A002,R051,02-00-01,00:00:00,REGULAR,-61468,-436253
2011-05-21,A002,R051,02-00-01,04:00:00,REGULAR,36,6
2011-05-21,A002,R051,02-00-01,08:00:00,REGULAR,15,11
